# OLS Parameter Estimation

## Imports

In [5]:
from IPython.display import display, Markdown
from loguru import logger
from tracks_import import read_from_csv
from typing import List
from scipy.linalg import lstsq

import pandas as pd
import numpy as np
import os

## Read in the data


In [6]:
# Specify the paths
dataset_dir = "../../datasets/inD/data/"
recording = "00"

if not os.path.isdir(dataset_dir):
    raise Exception("Directory does not exist")

# Create paths to csv files
tracks_file = dataset_dir + recording + "_tracks.csv"
tracks_meta_file = dataset_dir + recording + "_tracksMeta.csv"
recording_meta_file = dataset_dir + recording + "_recordingMeta.csv"

if not os.path.isfile(tracks_file) or not os.path.isfile(tracks_meta_file) or not os.path.isfile(recording_meta_file):
    raise Exception("File does not exist")
# Load csv files
logger.info("Loading csv files {}, {} and {}", tracks_file, tracks_meta_file, recording_meta_file)
tracks, static_info, meta_info = read_from_csv(tracks_file, tracks_meta_file, recording_meta_file,
                                                   include_px_coordinates=True)


2023-12-08 12:59:14.354 | INFO     | __main__:<module>:16 - Loading csv files ../../datasets/inD/data/00_tracks.csv, ../../datasets/inD/data/00_tracksMeta.csv and ../../datasets/inD/data/00_recordingMeta.csv


In [7]:
df_tracks = pd.DataFrame(tracks)
df_static_info = pd.DataFrame(static_info)
df_meta_info = pd.DataFrame(meta_info.items())


print("df_tracks")
display(df_tracks)

print("df_static_info")
display(df_static_info)

print("df_meta_info")
display(df_meta_info)


df_tracks


,recordingId,trackId,frame,trackLifetime,xCenter,yCenter,heading,width,length,xVelocity,...,latAcceleration,center,bbox,xCenterVis,yCenterVis,centerVis,widthVis,lengthVis,headingVis,bboxVis
0,0,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[131.38519, 131.3383, 131.28821, 131.23861, 13...","[-34.7838, -35.13993, -35.50286, -35.86783, -3...","[261.84968, 261.95002, 262.00456, 262.03306, 2...","[1.76028, 1.76028, 1.76028, 1.76028, 1.76028, ...","[4.03141, 4.03141, 4.03141, 4.03141, 4.03141, ...","[-1.20819, -1.20844, -1.20806, -1.20726, -1.20...",...,"[0.08452, 0.09037, 0.09908, 0.11096, 0.12583, ...","[[131.38519, -34.7838], [131.3383, -35.13993],...","[[[131.97067233266367, -36.90392346450948], [1...","[10345.343282535594, 10341.651137732073, 10337...","[2738.895850217681, 2766.937725433673, 2795.51...","[[10345.343282535594, 2738.895850217681], [103...","[138.60543089660072, 138.60543089660072, 138.6...","[317.435476271312, 317.435476271312, 317.43547...","[98.15032000000002, 98.04998, 97.9954399999999...","[[[10254.239037769681, 2886.1853724283237], [1..."
1,0,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[135.03907, 135.0677, 135.09607, 135.12432, 13...","[-45.99449, -46.1289, -46.26314, -46.39582, -4...","[275.68596, 275.99277, 276.32133, 276.67232, 2...","[1.84093, 1.84093, 1.84093, 1.84093, 1.84093, ...","[3.92792, 3.92792, 3.92792, 3.92792, 3.92792, ...","[0.72075, 0.72438, 0.72821, 0.73225, 0.73619, ...",...,"[-0.19502, -0.20565, -0.21602, -0.22585, -0.23...","[[135.03907, -45.99449], [135.0677, -46.1289],...","[[[136.149587332072, -47.85759116552825], [135...","[10633.05183563196, 10635.306177831251, 10637....","[3621.631845683296, 3632.215364195585, 3642.78...","[[10633.05183563196, 3621.631845683296], [1063...","[144.95585696620944, 144.95585696620944, 144.9...","[309.2866158380348, 309.2866158380348, 309.286...","[84.31403999999998, 84.00722999999999, 83.6786...","[[[10576.251934510436, 3782.6951066521556], [1..."
2,0,2,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[113.34668, 113.05249, 112.75417, 112.45299, 1...","[-65.72286, -65.90637, -66.09502, -66.28951, -...","[213.33247, 213.3922, 213.45667, 213.53202, 21...","[1.84183, 1.84183, 1.84183, 1.84183, 1.84183, ...","[4.26398, 4.26398, 4.26398, 4.26398, 4.26398, ...","[-7.36258, -7.43408, -7.50676, -7.58034, -7.65...",...,"[-0.63784, -0.64564, -0.64967, -0.64971, -0.64...","[[113.34668, -65.72286], [113.05249, -65.90637...","[[[112.07145015521381, -67.66380168904982], [1...","[8924.980924681935, 8901.816239679849, 8878.32...","[5175.054724280775, 5189.5044042315985, 5204.3...","[[8924.980924681935, 5175.054724280775], [8901...","[145.0267234691561, 145.0267234691561, 145.026...","[335.74816803831635, 335.74816803831635, 335.7...","[146.66753, 146.6078, 146.54333, 146.46798, 14...","[[[8744.876953231256, 5206.7162969361925], [90..."
3,0,3,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[89.96716, 89.92104, 89.87504, 89.83031, 89.78...","[-73.02951, -73.04865, -73.06778, -73.08882, -...","[207.02469, 207.02151, 207.02047, 207.02288, 2...","[3.06624, 3.06624, 3.06624, 3.06624, 3.06624, ...","[19.08486, 19.08486, 19.08486, 19.08486, 19.08...","[-1.16402, -1.13548, -1.1056, -1.07437, -1.041...",...,"[-0.12784, -0.13041, -0.13217, -0.13303, -0.13...","[[89.96716, -73.02951], [89.92104, -73.04865],...","[[[82.16327072838743, -78.73106588201239], [99...","[7084.0644547136935, 7080.432940140471, 7076.8...","[5750.384428453207, 5751.891522749205, 5753.39...","[[7084.0644547136935, 5750.384428453207], [708...","[241.4374511057292, 241.4374511057292, 241.437...","[1502.7525415850314, 1502.7525415850314, 1502....","[152.97531, 152.97849, 152.97953, 152.97712, 1...","[[[6359.878852612863, 5984.2530341253505], [76..."
4,0,4,"[0, 1, 2, 3, 4, 5, 

df_static_info


,recordingId,trackId,initialFrame,finalFrame,numFrames,width,length,class
0,0,0,0,277,278,1.76028,4.03141,car
1,0,1,0,526,527,1.84093,3.92792,car
2,0,2,0,129,130,1.84183,4.26398,car
3,0,3,0,1139,1140,3.06624,19.08486,truck_bus
4,0,4,0,24026,24027,1.82684,4.09254,car
...,...,...,...,...,...,...,...,...
379,0,379,23769,23981,213,1.82039,4.28429,car
380,0,380,23844,24027,184,2.84928,19.94480,truck_bus
381,0,381,23945,24026,82,1.95874,5.44133,car
382,0,382,23970,24026,57,1.90287,4.40178,car


df_meta_info


,0,1
0,recordingId,0
1,locationId,4
2,frameRate,25.0
3,speedLimit,13.88889
4,weekday,wednesday
5,startTime,16
6,duration,961.08
7,numTracks,384
8,numVehicles,324
9,numVRUs,60


## Calculate distance/velocity/acceleration
Position
- x-Position
- y-Position
- longitudinal-Distance

Velocity
- x-Velocity
- y-Velocity
- longitudinal-Velocity

Acceleration
- x-Acceleration
- y-Acceleration
- longitudinal-Acceleration



### Test if distances add up
Total distance: summed distance between the datapoints

Linear distance: distance between the starting and the endpoint

In [8]:
def calc_distance(x_prev, y_prev, x_cur, y_cur):
    x_delta = abs(x_cur - x_prev)
    y_delta = abs(y_cur - y_prev)
    return np.sqrt(x_delta ** 2 + y_delta ** 2)
# Take track 16 for reference
track_id = 16
track_current = tracks[track_id]

# Get position column
xPositions = track_current["xCenter"]
yPositions = track_current["yCenter"]

# Calculate distance
distance_total = 0

# Calculate distance dependent on time
distance_per_frame = []

x_prev, y_prev = xPositions[0], yPositions[0]

if len(xPositions) != len(yPositions):
    raise Exception("Position list don't match in length")
else:
    for x_cur, y_cur in zip(xPositions, yPositions):
        dist = calc_distance(x_prev, y_prev, x_cur, y_cur)
        x_prev = x_cur
        y_prev = y_cur
        distance_total += dist
        distance_per_frame.append(distance_total)

# Total distance
print("Total distance: " + str(distance_total))

# Linear distance
linear_distance = calc_distance( xPositions[0] , yPositions[0] , xPositions[len(xPositions)-1] , yPositions[len(yPositions)-1])
print("Linear distance: " + str(linear_distance))

Total distance: 122.49139053554025
Linear distance: 122.37336881019212


In [9]:
distance_per_frame

[0.0,
 0.592410991879794,
 1.1915115510516192,
 1.7952620407408606,
 2.4026963097774354,
 3.0138726851389297,
 3.628124475612499,
 4.243204782889537,
 4.860283277815419,
 5.478060250192996,
 6.094423260520512,
 6.709830178258684,
 7.323925101540093,
 7.937182062772496,
 8.550951651461933,
 9.163082376389209,
 9.77487884859219,
 10.386817405148316,
 10.996706361680573,
 11.605918681483768,
 12.213989526451698,
 12.8221786625763,
 13.430021388452003,
 14.037068656216213,
 14.645471445493195,
 15.252911047511367,
 15.86070497944883,
 16.46922083708936,
 17.075360133291152,
 17.6786718889366,
 18.28029808600585,
 18.881248867101945,
 19.482578817877716,
 20.08299276125346,
 20.683478360831,
 21.283945629297815,
 21.883983695506956,
 22.483694892667714,
 23.08234456559839,
 23.680765859754946,
 24.27907546564473,
 24.876491424085753,
 25.473932023392496,
 26.06900237730032,
 26.663569398249184,
 27.257340373120805,
 27.84952147734485,
 28.441561428192756,
 29.032678059746978,
 29.6244153197

### Calculate the velocity
-  v(k)
-  v(k-1)
-  v(k+1)
-  -v(k-2)


### Compare the velocity datapoints
Here we calculate the maximum difference between all the  x-y-velocities and the lat-long velocities

In [10]:
# Calculate the resulting velocity vector
def velocity_total(xVel, yVel):
    res = []
    for x, y in zip(xVel, yVel):
        res.append(np.sqrt(x**2 + y**2))
    
    return res
       
# Get velocity column
x_velocities = track_current["xVelocity"]
y_velocities = track_current["yVelocity"]

x_y_velocities= velocity_total(x_velocities, y_velocities)

lon_velocities = track_current["lonVelocity"]
lat_velocities = track_current["latVelocity"]

lat_lon_velocities = velocity_total(lon_velocities, lat_velocities)

# Just for testing the differences
def difference_list(a, b):
    res = []
    for x, y in zip(a,b):
        res.append(abs(x-y))
    return res


print("This is the maximum difference of the two velocities:")
max(difference_list(lat_lon_velocities, x_y_velocities))

This is the maximum difference of the two velocities:


1.0206503132437206e-05

In the following we will continue with x_y_velocities

## Perform OLS for the parameters $\overline\theta$
$$s_i(k+1) - s_i(k) =
\begin{bmatrix}
    s_i(k-1)
    & s_i(k-2)
    & v_i(k)
    & v_i(k-1)
    & v_i(k+1)
    & - v_i(k-2)
\end{bmatrix}
\begin{bmatrix}

    \overline{\theta_{s_1}}
    \\ \overline{\theta_{s_2}}
    \\ \overline{\theta_{s_3}} - \overline{\theta_{v_1}}
    \\ \overline{\theta_{s_4}} - \overline{\theta_{v_2}}
    \\ \overline{\theta_{v_4}}
    \\ \overline{\theta_{v_3}}
\end{bmatrix}$$

$$\begin{bmatrix}
    \overline{\theta_{s_1}}
    & \overline{\theta_{s_2}}
    & \overline{\theta_{s_3}} - \overline{\theta_{v_1}}
    & \overline{\theta_{s_4}} - \overline{\theta_{v_2}}
    & \overline{\theta_{v_4}}
    & \overline{\theta_{v_3}}
\end{bmatrix}

\begin{bmatrix}
    s_i(k-1)
    \\ s_i(k-2)
    \\ v_i(k)
    \\ v_i(k-1)
    \\ v_i(k+1)
    \\ - v_i(k-2)
\end{bmatrix}$$



In [11]:
# Create s and v matrix
s = np.array(distance_per_frame)
v = np.array(x_y_velocities)
s.shape

(211,)

In [12]:
v.shape

(211,)

In [13]:

# Create the design matrix A based on your formula
n = len(s)
A = np.vstack([
    s[1:n-2],          # s_i(k-1)
    s[0:n-3],          # s_i(k-2)
    v[2:n-1],          # v_i(k)
    v[1:n-2],          # v_i(k-1)
    v[3:n],            # v_i(k+1)
   -v[0:n-3]           # -v_i(k-2)
]).T

# Target variable (s(k+1) - s(k))
target = np.array(s[3:] - s[2:n-1])

# Shape of A matrix and target
print("Shape:")
print("A matrix: " + str(A.shape))
print("Target: " + str(target.shape))




Shape:
A matrix: (208, 6)
Target: (208,)


||t-Ax||^2



||t-Ax||^2 + lambda ||x||^2

Normalisation step:
 
|| [t 0 ]- diag (A, I*sqrt(lambda)) x || ^2


A
B

6   6

A   x
B   0


lambda is variable



In [14]:
# Perform OLS regression to estimate coefficients
coefficients, residuals, rank, s = lstsq(A, target)
# Parameters
s1, s2, ds3, ds4, v4, v3 =  coefficients


display(Markdown(f"""
**Estimated Parameters:**
- a | $\\overline\\theta_{{s_1}}$: {s1}
- b | $\\overline\\theta_{{s_2}}$: {s2}
- c | $\\overline{{\\overline\\theta}}{{s_3}}$: {ds3}
- d | $\\overline{{\\overline\\theta}}{{s_4}}$: {ds4}
- e | $\\overline\\theta_{{v_4}}$: {v4}
- f | $\\overline\\theta_{{v_3}}$: {v3}
"""))



**Estimated Parameters:**
- a | $\overline\theta_{s_1}$: 0.2987561002115432
- b | $\overline\theta_{s_2}$: -0.2987509460595542
- c | $\overline{\overline\theta}{s_3}$: 5.7800519946248
- d | $\overline{\overline\theta}{s_4}$: -5.012676853406745
- e | $\overline\theta_{v_4}$: -2.1519407399083224
- f | $\overline\theta_{v_3}$: -1.4125922655560528


### Calculate the parameters $\theta$


In [15]:
# Theta Coefficients
s1,s2,s3,s4,s5 = s1,s2,0,0,0
v1,v2,v3,v4 = 0,0,0,0

# Distance, speed, acceleration parameters
s_0, s_1, s_2 = 0,0,0
v_0, v_1, v_2 = 0,0,0
a = 0


## Linear regression for $\theta_{s_i}$

I am given all the datapoints for s(...) and v(...) and a(...).
Given this formula:
$$s_i(k+1) = s_i(k) + \theta_{s_1} s_i(k-1) + \theta_{s_2} s_i(k-2) + \theta_{s_3} v_i(k) + \theta_{s_4}v_i(k-1) + \theta_{s_5} a_i(k)$$

How can I estimate the theta parameters using python and machine learning



In [18]:
def calculate_distance(s, v, os, a):
    """
    s[k, k-1, k-2]
    v[k, k-1]
    a[k]
    os[s1, s2, s3, s4, s5]
    """
    s_next = s[0] + os[0]*s[1] + os[1]*s[2] + os[2]*v[0] + os[3]*v[1] + os[4]*a
    return s_next

def calculate_velocity(v, ov, a):
    """
    s[k, k-1, k-2]
    v[k, k-1]
    a[k]
    os[s1, s2, s3, s4, s5]
    """
 
    v_next = ov[0]*v[0] + ov[1]*v[1] + ov[2]*v[2] + ov[3]*a
    return v_next

def calculate_acceleration_s(s, v, os):
    """
    s[sk+1, sk, sk-1, sk-2]
    v[vk, vk-1]
    os[s1, s2, s3, s4, s5]
    """

    a = (s[0] - s[1] - os[0]*s[2] - os[1]*s[3] - os[2]*v[0] - os[3]*v[1]) / os[4]
    return a

def calculate_acceleration_v(v, ov):
    """
    v[k+1, k, k-1, k-2]
    ov[s1, s2, s3]
    """

    a = (v[0] - ov[0]*v[1] - ov[1]*v[2] - ov[2]*v[3]) / ov[3]
    return a


In [20]:
X = np.random.rand(100, 10)
X.shape

(100, 10)

In [21]:

# Assuming A and B are matrices
A = np.array([[1, 2, 3],
              [4, 5, 6]])

B = np.array([[7, 8, 9],
              [10, 11, 12]])

lambd = 0.1
# Create the block diagonal matrix
#result = np.block([[A, np.zeros_like(A)], [np.zeros_like(B), B]])
result = np.diag(np.concatenate([A, np.sqrt(lambd)]))

print("Resulting Diagonal Matrix:")
print(result)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 0 dimension(s)